<a href="https://colab.research.google.com/github/m99Tanishq/tuner/blob/main/keras_tuner_on_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to the Keras Tuner

## Overview

The Keras Tuner is a library that helps you pick the optimal set of hyperparameters for your TensorFlow program. The process of selecting the right set of hyperparameters for your machine learning (ML) application is called *hyperparameter tuning* or *hypertuning*.

Hyperparameters are the variables that govern the training process and the topology of an ML model. These variables remain constant over the training process and directly impact the performance of your ML program. Hyperparameters are of two types:
1. **Model hyperparameters** which influence model selection such as the number and width of hidden layers
2. **Algorithm hyperparameters** which influence the speed and quality of the learning algorithm such as the learning rate for Stochastic Gradient Descent (SGD) and the number of nearest neighbors for a k Nearest Neighbors (KNN) classifier

In this tutorial, you will use the Keras Tuner to perform hypertuning for an image classification application.

## Setup

In [ ]:
import tensorflow as tf
from tensorflow import keras

Install and import the Keras Tuner.

In [ ]:
!pip install -q -U keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 53.2 MB/s eta 0:00:00


In [ ]:
import keras_tuner as kt

Load the data.

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Instantiate the tuner and perform hypertuning

Instantiate the tuner to perform the hypertuning. The Keras Tuner has four tuners available - `RandomSearch`, `Hyperband`, `BayesianOptimization`, and `Sklearn`. In this tutorial, you use the [Hyperband](https://arxiv.org/pdf/1603.06560.pdf) tuner.

To instantiate the Hyperband tuner, you must specify the hypermodel, the `objective` to optimize and the maximum number of epochs to train (`max_epochs`).

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

Creating a callback to stop training early after reaching a certain value for the validation loss.

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Run the hyperparameter search. The arguments for the search method are the same as those used for `tf.keras.model.fit` in addition to the callback above.

In [ ]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 01m 22s]
val_accuracy: 0.874916672706604

Best val_accuracy So Far: 0.8944166898727417
Total elapsed time: 00h 12m 20s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 256 and the optimal learning rate for the optimizer
is 0.001.



## Train the model

Find the optimal number of epochs to train the model with the hyperparameters obtained from the search.

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.4975 - accuracy: 0.8238 - val_loss: 0.4012 - val_accuracy: 0.8539
Epoch 2/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3743 - accuracy: 0.8641 - val_loss: 0.3908 - val_accuracy: 0.8623
Epoch 3/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3363 - accuracy: 0.8777 - val_loss: 0.3669 - val_accuracy: 0.8653
Epoch 4/50
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3104 - accuracy: 0.8867 - val_loss: 0.3443 - val_accuracy: 0.8749
Epoch 5/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2890 - accuracy: 0.8928 - val_loss: 0.3200 - val_accuracy: 0.8860
Epoch 6/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2755 - accuracy: 0.8973 - val_loss: 0.3272 - val_accuracy: 0.8819
Epoch 7/50
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2621 - accuracy: 0.9027 - val_loss: 0.3126 - val_accuracy:

Re-instantiate the hypermodel and train it with the optimal number of epochs from above.

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/39
1500/1500 [==============================] - 4s 3ms/step - loss: 0.5064 - accuracy: 0.8210 - val_loss: 0.4136 - val_accuracy: 0.8503
Epoch 2/39
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3767 - accuracy: 0.8635 - val_loss: 0.3786 - val_accuracy: 0.8639
Epoch 3/39
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3383 - accuracy: 0.8752 - val_loss: 0.3342 - val_accuracy: 0.8806
Epoch 4/39
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3121 - accuracy: 0.8847 - val_loss: 0.3497 - val_accuracy: 0.8757
Epoch 5/39
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2940 - accuracy: 0.8921 - val_loss: 0.3228 - val_accuracy: 0.8842
Epoch 6/39
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2782 - accuracy: 0.8956 - val_loss: 0.3246 - val_accuracy: 0.8832
Epoch 7/39
1500/1500 [==============================] - 4s 3ms/step - loss: 0.2647 - accuracy: 0.8996 - val_loss: 0.3369 - val_accuracy:

Evaluating the hypermodel on the test data.

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 1s 2ms/step - loss: 0.5176 - accuracy: 0.8794
[test loss, test accuracy]: [0.5175815224647522, 0.8794000148773193]
